In [1]:
import os

# Rename ifgs with gap in 20230516 -- ONLY RUN THIS ONCE

In [ ]:
ifg_data_folder = '/uufs/chpc.utah.edu/common/home/lin-group15/agm/em27/ua/inst_data/ifgs'
date = '20230516'
date_folder = os.path.join(ifg_data_folder,date)

In [12]:
#Correct gap in ifg data
good_data_ifgs = []
no_data_ifgs = []

for file in sorted(os.listdir(date_folder)):
    fsize_kb = os.stat(os.path.join(date_folder,file)).st_size/1024
    if fsize_kb>2:
        good_data_ifgs.append(file)
    else:
        no_data_ifgs.append(file)

for file in no_data_ifgs:
    os.remove(os.path.join(date_folder,file))

In [31]:
before_3600_ids = []
after_3600_ids = []
for file in sorted(os.listdir(date_folder)):
    ifg_id =int(file.split('.')[-1])
    if ifg_id < 3601:
        before_3600_ids.append(ifg_id)
    else:
        after_3600_ids.append(ifg_id)
last_before_3600_id = max(before_3600_ids)

new_id = last_before_3600_id

for old_id in after_3600_ids:
    new_id = new_id+1
    new_id_str = f'{new_id:04}'

    old_fullname = os.path.join(date_folder,f'ua{date}.ifg.{old_id}')
    new_fullname = os.path.join(date_folder,f'ua{date}.ifg.{new_id_str}')
    os.rename(old_fullname,new_fullname)